In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
# Suppress just SettingWithCopyWarning
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Part 1: Getting current routes

Now, we get the current routes for each airport based on their wikipedia article based on a series of scripts developed in the first notebook. Save to a current_routes.csv file. 6


defining key functions

In [ ]:
#function to extract text between two found substrings
def extract_between(text, start, end):
    start_idx = text.find(start)
    if start_idx == -1:
        return "" #return empty string
    start_idx += len(start)
    end_idx = text.find(end, start_idx)
    if end_idx == -1:
        return ""
    return text[start_idx:end_idx]
#function to get iata code from wikipedia name
def get_iata_code(iata,ref_data=pd.read_csv("./data/top_airports_basic_data.csv")):
    matched = ref_data[ref_data["wiki_name"]==iata].iloc[0]
    return matched["iata"]
#custom function to find the text matching the passenger destination section 
def extract_passenger_destinations(text, start, end):
    start_idx = text.find(start)
    #narrow down search text 
    if start_idx == -1:
        return "" #return empty string
    start_idx += len(start)
    start_idx += 3 #increment by 3 to account for ==
    end_idx = text.find(end, start_idx)
    if end_idx == -1:
        return ""
    return text[start_idx:end_idx]

def get_destinations(iata_source, wiki_name, file_append_path):
    #open file in append mode
    f = open(file_append_path, "a")
    
    search_url = f"https://en.wikipedia.org/w/index.php?title={wiki_name}&action=raw"
    response = requests.get(search_url)
    full_text = response.text
    #get to the passenger destinations section marked by a predictiable substrings ()
    # Call both and choose the first non-None result
    destinations_text = extract_passenger_destinations(full_text, "Airlines and destinations","==") 
    #if the length if less than 5, a low amount, we try differently
    if("Airport" not in destinations_text): destinations_text = extract_passenger_destinations(full_text, "Airlines and destinations","Cargo") 
    destinations_list = destinations_text.split("<!-- -->")#split predictably
    
        #ignore the first part, since that is some disclaimer
    for i in range(1, len(destinations_list)):
        airlineDest = destinations_list[i]#we get a list relating to airlines and their list of destinations
        #get information before and the seasonal destination section (if present)
        beforeSeasonal = ""
        afterSeasonal = None
        if "'''Seasonal:'''" in airlineDest:
            seasonalSplit = airlineDest.split("'''Seasonal:'''")
            beforeSeasonal = seasonalSplit[0]
            afterSeasonal = seasonalSplit[1]
        else: 
            beforeSeasonal = airlineDest 
        
        #get the airline based on a predictable pattern (in the first element)
        beforeSeasonalsplit = beforeSeasonal.split(",") #split based on commas
        airline_firstdest = beforeSeasonalsplit[0].split(" | [[")  #split between the airline and first destination
        airline = str(airline_firstdest[0]) #extract the airline
        airline = extract_between(airline,"[[","]]") #remove the [[]]
        #do a final split just in case (even if | is not found), [0] selects the current string
        firstdest = ""  #get first destination (assuming not seasonal) 
        if len(airline_firstdest) == 2:
            destination = airline_firstdest[1].split("|")[0].strip()
            destination_wiki_name = destination.replace(" ", "_") #replace destination to get the destination iata_code
            f.write(f"{iata_source},{wiki_name},{destination_wiki_name},{airline},0\n") #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
        #loop thorugh the rest of the comma list for the before seasonal split if possible 
        for i in range(1, len(beforeSeasonalsplit)):
            destination = beforeSeasonalsplit[i]
            extracted_destination = extract_between(destination,"[[","]]")
            if (extracted_destination!=""): #assuming 
                destination_wiki_name = extracted_destination.split("|")[0].strip()
                destination_wiki_name = destination_wiki_name.replace(" ", "_")
                f.write(f"{iata_source},{wiki_name},{destination_wiki_name},{airline},0\n") #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
        #loop through the seasonal destinations list if the current airline has it
        if (afterSeasonal):
            afterSeasonalSplit = afterSeasonal.split(",")
            for i in range(0, len(afterSeasonalSplit)):
                destination = afterSeasonalSplit[i]
                extracted_destination = extract_between(destination,"[[","]]")
                if (extracted_destination!=""): #assuming 
                    destination_wiki_name = extracted_destination.split("|")[0].strip()
                    destination_wiki_name = destination_wiki_name.replace(" ", "_")
                    f.write(f"{iata_source},{wiki_name},{destination_wiki_name},{airline},1\n") #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
    f.close() #close file at end
    return "done"

Start the file to store modern route data

In [3]:
f = open("./data/current_routes.csv", "w")
f.write("iata_source,starting_wiki_name,dest_wikipedia_name,airline,isSeasonal") 
f.close() #add column names

get iata code, wikiname lists

In [4]:
data = pd.read_csv("./data/top_airports_basic_data.csv")
print(len(data))
data.head(n=1)
codes_list = data["iata"]
names_list = data["wiki_name"]
file_append_path = "./data/current_routes.csv"

989


iterrate through airports 0 to 50

In [5]:
for i in range(0,50):
    print("airport index:", i)
    code = codes_list[i]
    name = names_list[i]
    get_destinations(code,name,file_append_path)
    

airport index: 0
airport index: 1
airport index: 2
airport index: 3
airport index: 4
airport index: 5
airport index: 6
airport index: 7
airport index: 8
airport index: 9
airport index: 10
airport index: 11
airport index: 12
airport index: 13
airport index: 14
airport index: 15
airport index: 16
airport index: 17
airport index: 18
airport index: 19
airport index: 20
airport index: 21
airport index: 22
airport index: 23
airport index: 24
airport index: 25
airport index: 26
airport index: 27
airport index: 28
airport index: 29
airport index: 30
airport index: 31
airport index: 32
airport index: 33
airport index: 34
airport index: 35
airport index: 36
airport index: 37
airport index: 38
airport index: 39
airport index: 40
airport index: 41
airport index: 42
airport index: 43
airport index: 44
airport index: 45
airport index: 46
airport index: 47
airport index: 48
airport index: 49


iterrate through airports 50 to 100

In [6]:
for i in range(50,100):
    print("airport index:", i)
    code = codes_list[i]
    name = names_list[i]
    get_destinations(code,name,file_append_path)

airport index: 50
airport index: 51
airport index: 52
airport index: 53
airport index: 54
airport index: 55
airport index: 56
airport index: 57
airport index: 58
airport index: 59
airport index: 60
airport index: 61
airport index: 62
airport index: 63
airport index: 64
airport index: 65
airport index: 66
airport index: 67
airport index: 68
airport index: 69
airport index: 70
airport index: 71
airport index: 72
airport index: 73
airport index: 74
airport index: 75
airport index: 76
airport index: 77
airport index: 78
airport index: 79
airport index: 80
airport index: 81
airport index: 82
airport index: 83
airport index: 84
airport index: 85
airport index: 86
airport index: 87
airport index: 88
airport index: 89
airport index: 90
airport index: 91
airport index: 92
airport index: 93
airport index: 94
airport index: 95
airport index: 96
airport index: 97
airport index: 98
airport index: 99


iterate through airports 100 to 150

In [7]:
for i in range(100, 150):
    print("airport index:", i)
    code = codes_list[i]
    name = names_list[i]
    get_destinations(code,name,file_append_path)

airport index: 100
airport index: 101
airport index: 102
airport index: 103
airport index: 104
airport index: 105
airport index: 106
airport index: 107
airport index: 108
airport index: 109
airport index: 110
airport index: 111
airport index: 112
airport index: 113
airport index: 114
airport index: 115
airport index: 116
airport index: 117
airport index: 118
airport index: 119
airport index: 120
airport index: 121
airport index: 122
airport index: 123
airport index: 124
airport index: 125
airport index: 126
airport index: 127
airport index: 128
airport index: 129
airport index: 130
airport index: 131
airport index: 132
airport index: 133
airport index: 134
airport index: 135
airport index: 136
airport index: 137
airport index: 138
airport index: 139
airport index: 140
airport index: 141
airport index: 142
airport index: 143
airport index: 144
airport index: 145
airport index: 146
airport index: 147
airport index: 148
airport index: 149


iterate through airports 150 to 200

In [10]:
for i in range(150,200):
    print("airport index:", i)
    code = codes_list[i]
    name = names_list[i]
    get_destinations(code,name,file_append_path)

airport index: 149
airport index: 150
airport index: 151
airport index: 152
airport index: 153
airport index: 154
airport index: 155
airport index: 156
airport index: 157
airport index: 158
airport index: 159
airport index: 160
airport index: 161
airport index: 162
airport index: 163
airport index: 164
airport index: 165
airport index: 166
airport index: 167
airport index: 168
airport index: 169
airport index: 170
airport index: 171
airport index: 172
airport index: 173
airport index: 174
airport index: 175
airport index: 176
airport index: 177
airport index: 178
airport index: 179
airport index: 180
airport index: 181
airport index: 182
airport index: 183
airport index: 184
airport index: 185
airport index: 186
airport index: 187
airport index: 188
airport index: 189
airport index: 190
airport index: 191
airport index: 192
airport index: 193
airport index: 194
airport index: 195
airport index: 196
airport index: 197
airport index: 198
airport index: 199


iterate through airports 200 to 250

In [11]:
for i in range(200,250):
    print("airport index:", i)
    code = codes_list[i]
    name = names_list[i]
    get_destinations(code,name,file_append_path)

airport index: 200
airport index: 201
airport index: 202
airport index: 203
airport index: 204
airport index: 205
airport index: 206
airport index: 207
airport index: 208
airport index: 209
airport index: 210
airport index: 211
airport index: 212
airport index: 213
airport index: 214
airport index: 215
airport index: 216
airport index: 217
airport index: 218
airport index: 219
airport index: 220
airport index: 221
airport index: 222
airport index: 223
airport index: 224
airport index: 225
airport index: 226
airport index: 227
airport index: 228
airport index: 229
airport index: 230
airport index: 231
airport index: 232
airport index: 233
airport index: 234
airport index: 235
airport index: 236
airport index: 237
airport index: 238
airport index: 239
airport index: 240
airport index: 241
airport index: 242
airport index: 243
airport index: 244
airport index: 245
airport index: 246
airport index: 247
airport index: 248
airport index: 249


iterate through airports 250 to 300

In [12]:
for i in range(250,300):
    print("airport index:", i)
    code = codes_list[i]
    name = names_list[i]
    get_destinations(code,name,file_append_path)

airport index: 250
airport index: 251
airport index: 252
airport index: 253
airport index: 254
airport index: 255
airport index: 256
airport index: 257
airport index: 258
airport index: 259
airport index: 260
airport index: 261
airport index: 262
airport index: 263
airport index: 264
airport index: 265
airport index: 266
airport index: 267
airport index: 268
airport index: 269
airport index: 270
airport index: 271
airport index: 272
airport index: 273
airport index: 274
airport index: 275
airport index: 276
airport index: 277
airport index: 278
airport index: 279
airport index: 280
airport index: 281
airport index: 282
airport index: 283
airport index: 284
airport index: 285
airport index: 286
airport index: 287
airport index: 288
airport index: 289
airport index: 290
airport index: 291
airport index: 292
airport index: 293
airport index: 294
airport index: 295
airport index: 296
airport index: 297
airport index: 298
airport index: 299


iterate through airports 300 to 350

In [13]:
for i in range(300,350):
    print("airport index:", i)
    code = codes_list[i]
    name = names_list[i]
    get_destinations(code,name,file_append_path)

airport index: 300
airport index: 301
airport index: 302
airport index: 303
airport index: 304
airport index: 305
airport index: 306
airport index: 307
airport index: 308
airport index: 309
airport index: 310
airport index: 311
airport index: 312
airport index: 313
airport index: 314
airport index: 315
airport index: 316
airport index: 317
airport index: 318
airport index: 319
airport index: 320
airport index: 321
airport index: 322
airport index: 323
airport index: 324
airport index: 325
airport index: 326
airport index: 327
airport index: 328
airport index: 329
airport index: 330
airport index: 331
airport index: 332
airport index: 333
airport index: 334
airport index: 335
airport index: 336
airport index: 337
airport index: 338
airport index: 339
airport index: 340
airport index: 341
airport index: 342
airport index: 343
airport index: 344
airport index: 345
airport index: 346
airport index: 347
airport index: 348
airport index: 349


iterate through airports 350 to 400

In [15]:
for i in range(350, len(codes_list)):
    print("airport index:", i)
    code = codes_list[i]
    name = names_list[i]
    get_destinations(code,name,file_append_path)

airport index: 350
airport index: 351
airport index: 352
airport index: 353
airport index: 354
airport index: 355
airport index: 356
airport index: 357
airport index: 358
airport index: 359
airport index: 360
airport index: 361
airport index: 362
airport index: 363
airport index: 364
airport index: 365
airport index: 366
airport index: 367
airport index: 368
airport index: 369
airport index: 370
airport index: 371
airport index: 372
airport index: 373
airport index: 374
airport index: 375
airport index: 376
airport index: 377
airport index: 378
airport index: 379
airport index: 380
airport index: 381
airport index: 382
airport index: 383
airport index: 384
airport index: 385
airport index: 386
airport index: 387
airport index: 388
airport index: 389
airport index: 390
airport index: 391
airport index: 392
airport index: 393
airport index: 394
airport index: 395
airport index: 396
airport index: 397
airport index: 398
airport index: 399
airport index: 400
airport index: 401
airport inde

### checking missing airports